<a href="https://colab.research.google.com/github/sands-eg/GEE_training/blob/main/Dubai_EGS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geehydro # Life saver for plotting GEE stuff with Python!
!pip install pprint
!pip install geemap

import pprint
import ee
import folium
import geehydro
import geemap
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import pandas as pd

  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10141 sha256=5df9b99acb24f71f4699a82f9762e628933e7af017551fd3df8fd95475ada563
  Stored in directory: /root/.cache/pip/wheels/49/6c/92/019aeb37663078e6666afa9dd336e2d8df72d5683d7d357697
Successfully built geehydro
ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint
     |████████████████████████████████| 1.9 MB 24.8 MB/s 
     |████████████████████████████████| 8.6 MB 38.9 MB/s 
     |████████████████████████████████| 1.3 MB 33.6 MB/s 
     |████████████████████████████████| 1.2 MB 34.2 MB/s 
     |████████████████████████████████| 1.6 MB 40.4 MB/s 
     |████████████████████████████████| 98 kB 8.7 MB/s 
     |████████████████████████████████| 95 kB 3.9 MB/s 
     |████████████████████████████████| 93 kB 1.6 MB/s 
     |████████████████████████████████| 76 kB 5.8 MB/s 
     |████████████████████████████████| 216 kB 57.7 

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=R2bN82sBDnH7CFN5HH6oOOIKgXdZZopQ0JF3cO9UeYY&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWh-4Azmxb8dK_Af-PV2YT6dHy8gRe7f4o8ZtCp3yBXx8-7GiQKv8xw

Successfully saved authorization token.


In [ ]:
# REGION = ee.Geometry.Rectangle(54.8, 24.95, 55.7, 25.5)
# LON_CENTER = 55
# LAT_CENTER = 25

# landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

# # get collection of landsat images from 2013-2021 for June-August
# landsat = (landsat
#            .filterBounds(REGION)
#            #.filterDate('2013-01-01', '2021-12-31')
#            .filter(ee.Filter.calendarRange(6, 8, "month"))
#            #.filterMetadata('CLOUD_COVER', 'less_than', 50)
#            )
# landsat.size().getInfo()

155

In [3]:
AOI = ee.Geometry.Rectangle(54.8, 24.75, 55.6, 25.4)

# Landsat 8 surface re
dataset = (ee
           .ImageCollection("LANDSAT/LC08/C02/T1_L2")
           .filterBounds(AOI)
           .filter(ee.Filter.lt('CLOUD_COVER', 10))
           .filter(ee.Filter.calendarRange(6, 8, "month"))
          #  .filterDate('2013-01-01', '2013-12-31')
)
print(dataset.size().getInfo())

# visual correction for landsat 8 data
def applyScaleFactors(image): 
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return (image.addBands(opticalBands, None, True)
              .addBands(thermalBand, None, True)
              .addBands(image.metadata('system:time_start')))

dataset = dataset.map(applyScaleFactors)

126


In [4]:
def calc_yearly_composite(year):
  filtered = dataset.filter(ee.Filter.calendarRange(year, year, 'year'))
  # instead of returning the mean over all images one can use quality mosaic to 
  # return the most recent pixel that is unmasked (meaning cloud-free)
  return filtered.qualityMosaic('system:time_start') 

years = ee.List.sequence(2013, 2021)
dataset_ann_L8 = ee.ImageCollection.fromImages(years.map(calc_yearly_composite))
print(dataset_ann_L8.size().getInfo())

9


In [6]:
image = dataset_ann_L8.first().clip(AOI)  
# mosaic = dataset.mosaic().clip(AOI)
# ndvi = image.normalizedDifference(["SR_B4", "SR_B3"])
# viz_params_ndvi = {
#   'min': -1,
#   'max': 1,
#   'palette': ["blue", "white", "green"]
# }
viz_params_rgb = {
  "bands": ['SR_B4', 'SR_B3', 'SR_B2'],
  "min": 0.0,
  "max": 0.3,
}

LON_CENTER = 55.2
LAT_CENTER = 25.1

# Create a default map
Map = geemap.Map()
Map.setCenter(LON_CENTER, LAT_CENTER, 10)
Map.addLayer(image, viz_params_rgb, 'RGB')
# Map.addLayer(mosaic, viz_params_rgb, 'summer mosaic')
#Map.addLayer(ndvi, viz_params_ndvi, 'NDVI')

Map

Map(center=[25.1, 55.2], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

In [7]:
print(dataset_ann_L8.limit(15).getVideoThumbURL({
  'dimensions': 768,
  "region": AOI,
  'crs': 'EPSG:4326',
  "framesPerSecond": 1,
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 0,
  'max': 0.3
  }))

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/868d54773b6e44817191d166038c0a26-bbc837c5e5d7228b1e79a3681e49fd34:getPixels


In [ ]:
# Landsat 7 surface reflectance 
dataset = (ee
           .ImageCollection("LANDSAT/LE07/C02/T1_L2")
           .filterBounds(AOI)
           .filter(ee.Filter.lt('CLOUD_COVER', 10))
           .filter(ee.Filter.calendarRange(6, 8, "month"))
          #  .filterDate('2013-01-01', '2013-12-31')
)
print(dataset.size().getInfo())

# visual correction for landsat 7 data
def applyScaleFactors(image): 
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)
  return (image.addBands(opticalBands, None, True)
              .addBands(thermalBand, None, True)
              .addBands(image.metadata('system:time_start')))

dataset = dataset.map(applyScaleFactors)

186


In [ ]:
years = ee.List.sequence(2000, 2014) # 1999 no data?
dataset_ann_L7 = ee.ImageCollection.fromImages(years.map(calc_yearly_composite))
print(dataset_ann_L7.size().getInfo())

15


In [ ]:
image = dataset_ann_L7.first().clip(AOI)  
# mosaic = dataset.mosaic().clip(AOI)
# ndvi = image.normalizedDifference(["SR_B4", "SR_B3"])
# viz_params_ndvi = {
#   'min': -1,
#   'max': 1,
#   'palette': ["blue", "white", "green"]
# }
viz_params_rgb = {
  "bands": ['SR_B3', 'SR_B2', 'SR_B1'],
  "min": 0.0,
  "max": 0.3,
}
# Create a default map
Map = geemap.Map()
Map.setCenter(LON_CENTER, LAT_CENTER, 10)
Map.addLayer(image, viz_params_rgb, 'RGB')
# Map.addLayer(mosaic, viz_params_rgb, 'summer mosaic')
#Map.addLayer(ndvi, viz_params_ndvi, 'NDVI')

Map

Map(center=[25, 55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [ ]:
print(dataset_ann_L7.limit(15).getVideoThumbURL({
  'dimensions': 768,
  "region": AOI,
  'crs': 'EPSG:4326',
  "framesPerSecond": 1,
  'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
  'min': 0,
  'max': 0.3
  }))

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/0b21422eb088dba5f736982ad9ee9012-338e5ea74dba7f9867458267d6f582a7:getPixels


### Long-term Landsat TOA time series (decadal)

# import Landsat 4

In [54]:
def applyScaleFactors(image): 
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)
  return (image.addBands(opticalBands, None, True)
              .addBands(thermalBand, None, True))

def get_image_LS4(yyyy):
  ic = ee.ImageCollection('LANDSAT/LT04/C02/T1_L2')
  ic = ic.filterBounds(AOI)
  ic = ic.filterDate('{0}-01-01'.format(yyyy), '{0}-12-31'.format(yyyy))
  # ic = ic.filter(ee.Filter.calendarRange(6, 8, "month"))
  ic = ic.filterMetadata('CLOUD_COVER', 'less_than', 5)
  print(ic.size().getInfo())
  im = ic.mosaic()
  im = applyScaleFactors(im)
  return im.clip(AOI)

In [36]:
test_ls = ee.ImageCollection('LANDSAT/LT04/C02/T1_L2').filterBounds(AOI)
print(test_ls.size().getInfo())

28


In [61]:
# visualise
LS4 = get_image_LS4(1988)

# test_ls = ee.ImageCollection('LANDSAT/LM03/C01/T1').filterBounds(AOI).first()

viz_params_rgb = {
  "bands": ['SR_B3', 'SR_B2', 'SR_B1'],
  "min": 0.0,
  "max": 0.3,
}
# Create a default map
Map = geemap.Map()
Map.setCenter(LON_CENTER, LAT_CENTER, 10)
# Map.addLayer(test_ls, viz_params_rgb, 'test')
Map.addLayer(LS4, viz_params_rgb, 'LS4')
#Map.addLayer(ndvi, viz_params_ndvi, 'NDVI')

Map

5


Map(center=[25.1, 55.2], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

# Landsat 5

In [123]:
def applyScaleFactors(image): 
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.3) # original -0.2
  return (image.addBands(opticalBands, None, True))

def get_image_LS5(yyyy):
  ic = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
  ic = ic.filterBounds(AOI)
  ic = ic.filterDate('{0}-01-01'.format(yyyy), '{0}-12-31'.format(yyyy))
  # ic = ic.filter(ee.Filter.calendarRange(6, 8, "month"))
  ic = ic.filterMetadata('CLOUD_COVER', 'less_than', 5)
  print(ic.size().getInfo())
  im = ic.mosaic()
  im = applyScaleFactors(im)
  return im.clip(AOI)

  # modified scale factors:
def applyScaleFactorsb(image): 
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.29) # original -0.2
  return (image.addBands(opticalBands, None, True))

def get_image_LS5b(yyyy):
  ic = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
  ic = ic.filterBounds(AOI)
  ic = ic.filterDate('{0}-01-01'.format(yyyy), '{0}-12-31'.format(yyyy))
  # ic = ic.filter(ee.Filter.calendarRange(6, 8, "month"))
  ic = ic.filterMetadata('CLOUD_COVER', 'less_than', 5)
  print(ic.size().getInfo())
  im = ic.mosaic()
  im = applyScaleFactorsb(im)
  return im.clip(AOI)

In [115]:
# visualise
LS5 = get_image_LS5b(2008)

# test_ls = ee.ImageCollection('LANDSAT/LM03/C01/T1').filterBounds(AOI).first()

viz_params_rgb = {
  "bands": ['SR_B3', 'SR_B2', 'SR_B1'],
  "min": 0.0,
  "max": 0.3,
}
# Create a default map
Map = geemap.Map()
Map.setCenter(LON_CENTER, LAT_CENTER, 10)
# Map.addLayer(test_ls, viz_params_rgb, 'test')
Map.addLayer(LS5, viz_params_rgb, 'LS5')
#Map.addLayer(ndvi, viz_params_ndvi, 'NDVI')

Map

7


Map(center=[25.1, 55.2], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

# Landsat 8

In [117]:
def applyScaleFactors(image): 
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.22)
  # thermalBand = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return (image.addBands(opticalBands, None, True))

def get_image_LS8(yyyy):
  ic = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
  ic = ic.filterBounds(AOI)
  ic = ic.filterDate('{0}-01-01'.format(yyyy), '{0}-12-31'.format(yyyy))
  ic = ic.filter(ee.Filter.calendarRange(6, 8, "month"))
  ic = ic.filterMetadata('CLOUD_COVER', 'less_than', 5)
  print(ic.size().getInfo())
  im = ic.mosaic()
  im = applyScaleFactors(im)
  return im.clip(AOI)

In [140]:
# visualise
LS8 = get_image_LS8(2021)

# test_ls = ee.ImageCollection('LANDSAT/LM03/C01/T1').filterBounds(AOI).first()

viz_params_rgb = {
  "bands": ['SR_B4', 'SR_B3', 'SR_B2'],
  "min": 0.0,
  "max": 0.3,
}
# Create a default map
Map = geemap.Map()
Map.setCenter(LON_CENTER, LAT_CENTER, 10)
# Map.addLayer(test_ls, viz_params_rgb, 'test')
Map.addLayer(LS8.clip(site), viz_params_rgb, 'LS8')
#Map.addLayer(ndvi, viz_params_ndvi, 'NDVI')

Map

15


Map(center=[25.1, 55.2], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

In [106]:
# visualise
LS8 = get_image_LS8(2021).select(['SR_B4', 'SR_B3', 'SR_B2'])

# test_ls = ee.ImageCollection('LANDSAT/LM03/C01/T1').filterBounds(AOI).first()

viz_params_rgb = {
  # "bands": ['SR_B4', 'SR_B3', 'SR_B2'],
  "min": 0.0,
  "max": 0.3,
}
# Create a default map
Map = geemap.Map()
Map.setCenter(LON_CENTER, LAT_CENTER, 10)
# Map.addLayer(test_ls, viz_params_rgb, 'test')
Map.addLayer(LS8, viz_params_rgb, 'LS8')
#Map.addLayer(ndvi, viz_params_ndvi, 'NDVI')

Map

15


Map(center=[25.1, 55.2], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

## Combine images

In [124]:
# collect images:
LS5_1987 = get_image_LS5(1987).select(['SR_B3', 'SR_B2', 'SR_B1']) # (adapted scale factors -0.3 otherwise too bright)
LS5_1992 = get_image_LS5(1992).select(['SR_B3', 'SR_B2', 'SR_B1']) # as above
LS5_1997 = get_image_LS5(1997).select(['SR_B3', 'SR_B2', 'SR_B1']) # (adapted scale factors -0.3 otherwise too bright?)
LS5_2002 = get_image_LS5b(2002).select(['SR_B3', 'SR_B2', 'SR_B1']) # as above
LS5_2008 = get_image_LS5b(2008).select(['SR_B3', 'SR_B2', 'SR_B1']) # (adapted scale factors -0.3 otherwise too bright?)
LS8_2014 = get_image_LS8(2014).select(['SR_B4', 'SR_B3', 'SR_B2']) # (adapted scale factors -0.25 to match)
LS8_2021 = get_image_LS8(2021).select(['SR_B4', 'SR_B3', 'SR_B2']) # (adapted scale factors -0.25 to match)


lt_ts = ee.ImageCollection.fromImages([LS5_1987, LS5_1992, LS5_1997, LS5_2002, LS5_2008, LS8_2014, LS8_2021])
# LS4_1988 = get_image_LS4(1988) 

37
23
35
9
7
15
15


In [143]:
site = ee.Geometry.Rectangle(54.9, 24.8, 55.5, 25.4)

print(LS5_1997.getDownloadURL(params={
    "bands": ['SR_B3', 'SR_B2', 'SR_B1'],
    "region": site,
    "scale": 60,
    "format": 'GEO_TIFF'
  }))

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/8d06ea4634e6fbf049e6d06c189e1b3d-0e615691ae1a528aa97abf7bb1b1c775:getPixels


In [127]:
# time series 1987-2021 ~every 5 years 
print(lt_ts.limit(15).getVideoThumbURL({
  'dimensions': 768,
  "region": AOI,
  'crs': 'EPSG:4326',
  "framesPerSecond": 0.5,
  'min': 0,
  'max': 0.5
  }))

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/608b34f37f10369749dd455b9474cc1d-9c32acb8fa2aa4a18030b5c1606e609e:getPixels


In [ ]:
# import data 

# Applies scaling factors.
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True).addBands(thermalBand, None, True)

# Get image collections for modis and landsat
#def get_image_LS5(yyyy):
years = np.arange(start=1984,stop=2011)
def get_image_LS5(yyyy):
#for yyyy in years:
  p = ee.Geometry.Point(LON_CENTER,LAT_CENTER)
  ic = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
  ic = ic.filterBounds(REGION)
  ic = ic.filterDate('{0}-01-01'.format(yyyy), '{0}-12-31'.format(yyyy))
  ic = ic.filterMetadata('CLOUD_COVER', 'less_than', 5)
  print(ic.size().getInfo())
  im = ic.mosaic()
  im = applyScaleFactors(im)
  return im.clip(REGION)